# Visualising the label transfer 
> This module provides functionality to visualise the transfer of labels to interpret the results and how effective it was.

In [ ]:
#| default_exp viz

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score

def map_label_transfers(query_dataset, reference_label='type', predicted_query_label='annot'):
    """
    Maps the label transfers in the query dataset. This function takes into account that the label sets may not necessarily match.

    Args:
        query_dataset (AnnData): Query dataset with annotations.
        reference_label (str): Column name in query_dataset representing the original labels.
        predicted_query_label (str): Column name in query_dataset representing the transferred labels.
    
    Returns:
        label_transfers (DataFrame): A dataframe showing the proportion of each transferred label per original label.
    """
    # Check if the query_dataset is empty

    # Calculate the count of each transferred label per original label
    df = query_dataset.obs.groupby([reference_label, predicted_query_label])[reference_label].count().unstack().fillna(0).astype(int)
    
    
    # Replace any NaN values with 0, as these represent labels that did not match
    df = df.fillna(0)

    # Calculate the proportion of each transferred label per original label
    label_transfers = df.div(df.sum(axis=1), axis=0)

    # Check if only two cell types are showing up
    return label_transfers


def calculate_fuzz_1(query_dataset, reference_label='type', predicted_query_label='annot'):
    """
    Calculates the Fuzz1 score, which is the harmonic mean of precision and recall (akin to the F1 score).

    Args:
        query_dataset (DataFrame or similar): Query dataset with annotations.
        reference_label (str): Column name in query_dataset representing the original labels.
        predicted_query_label (str): Column name in query_dataset representing the annotations.
    
    Returns:
        fuzz1_score (float): The calculated Fuzz1 score.
    """
    # Compute precision and recall for each label
    labels = query_dataset.obs[reference_label].unique()
    precision = precision_score(query_dataset.obs[reference_label], query_dataset.obs[predicted_query_label], labels=labels, average='macro')
    recall = recall_score(query_dataset.obs[reference_label], query_dataset.obs[predicted_query_label], labels=labels, average='macro')
    
    # Compute the Fuzz1 score
    fuzz1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

    return fuzz1_score

import matplotlib.pyplot as plt
import seaborn as sns
import itertools

def visualize_label_distribution(proportions, fuzz1_score=None, title='label transfer distribution', proportions_flag=None,  matched = False):
    """
    Visualizes the distribution of annotations using a heatmap.

    Args:
        proportions (DataFrame): The proportions of each annotation per label.
        fuzz1_score (float, optional): The calculated Fuzz1 score. If specified, it will be included in the visualization.
        title (str): The title of the heatmap.
        proportions_flag (str, optional): Controls the display of proportions. Use 'none' for no annotations,
                                          'md' for main diagonal, or 'all' for all cells.
    
    Returns:
        heatmap (object): The heatmap object.
    """
    # Store the original figsize
    original_figsize = plt.rcParams['figure.figsize']
    
    # Set the figsize for this plot

    plt.rcParams['figure.figsize'] = (10, 10)
    
    # Sort the dataframe if all labels from predicted_query_label match the reference_label
    if matched == True:
        all_labels = np.union1d(proportions.index.values, proportions.columns.values)

        # Create a new DataFrame with zeros
        aligned_df = pd.DataFrame(data=np.zeros((len(all_labels), len(all_labels))),
                                index=all_labels, columns=all_labels)

        # Populate the new DataFrame with existing values
        for label in proportions.index:
            for cell_type in proportions.columns:
                if label in aligned_df.index and cell_type in aligned_df.columns:
                    aligned_df.at[label, cell_type] = proportions.at[label, cell_type]

        # If all labels match, sort the dataframe
        labels = proportions.index.intersection(proportions.columns)
        proportions_sorted = proportions.reindex(index=labels, columns=labels)
    else:
        # If they don't all match, use the original proportions
        proportions_sorted = proportions
    
    # Create the heatmap
    ax = sns.heatmap(proportions_sorted, cmap='viridis', annot=False, square=True, cbar_kws={'label': 'Proportion'})
    ax.set_title(title, fontsize=20)
    
    # Annotate cells based on the flag provided
    if proportions_flag == 'md':
        for i in range(len(proportions_sorted)):
            text = f"{proportions_sorted.iloc[i, i]:.2f}" if i < len(proportions_sorted.columns) else ""
            ax.text(i + 0.5, i + 0.5, text, ha='center', va='center', color='white' if proportions_sorted.iloc[i, i] < 0.5 else 'black', fontsize=10)
    elif proportions_flag == 'all':
        for i, j in itertools.product(range(len(proportions_sorted)), range(len(proportions_sorted.columns))):
            text = f"{proportions_sorted.iloc[i, j]:.2f}"
            ax.text(j + 0.5, i + 0.5, text, ha='center', va='center', color='white' if proportions_sorted.iloc[i, j] < 0.5 else 'black', fontsize=10)
    
    # Display the Fuzz1 score if it's provided
    if fuzz1_score is not None:
        plt.gcf().text(0.125, 0.9, f'Fuzz1 score: {fuzz1_score:.2f}', fontsize=12, bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=0.5'))
    
    # Show the plot
    plt.show()
    
    # Reset the figsize to the original
    plt.rcParams['figure.figsize'] = original_figsize
    
    # Reset seaborn to default settings
    sns.reset_defaults()
    
    # Return the Axes object containing the heatmap
    return ax



In [ ]:
#| export

import graphviz
from IPython.display import Image

def type_map_graph(label_alias_dict, type_flag='png'):
    """
    Creates and displays a graph visualization of cell type categorization.

    Parameters:
    - label_alias_dict: A dictionary mapping specific cell types to broader categories.
    - type_flag: The format for rendering ('png', 'svg', etc.), default is 'png'.
    """
    # Reverse the dictionary to map categories to cell types (subcategories)
    category_to_subcategories = {}
    for subcategory, category in label_alias_dict.items():
        if category not in category_to_subcategories:
            category_to_subcategories[category] = []
        category_to_subcategories[category].append(subcategory)

    # Create a new directed graph
    dot = graphviz.Digraph(comment='Cell Type Categorization', format=type_flag)

    # Adding nodes and subnodes to make it more readable
    for category, subcategories in category_to_subcategories.items():
        with dot.subgraph(name=f'cluster_{category}') as c:
            c.attr(style='filled', color='lightgrey')
            c.node_attr.update(style='filled', color='white')
            c.attr(label=category)
            for subcategory in subcategories:
                c.node(subcategory, label=subcategory)

    # Render the graph to a file in the specified format and display it
    dot.render('cell_type_categorization', format=type_flag, cleanup=True)
    display(Image('cell_type_categorization.' + type_flag))


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()